In [1]:
%pip install --upgrade -q transformers==4.53.3
%pip install einops timm datasets huggingface_hub


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Using cached pyarrow-21.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-21.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (42.7 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 21.0.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import os
import pandas as pd
import re
from PIL import Image
import random
from datasets import load_dataset
from huggingface_hub import login
from pathlib import Path
import os
import zipfile

In [12]:
%pip install --user --upgrade pyarrow==16.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 97.0 MB/s  0:00:006m0:00:010:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.2.0 requires pyarrow>=21.0.0, but you have pyarrow 16.1.0 which is incompatible.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 16.1.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import sys
import os

user_site = os.path.expanduser("~/.local/lib/python3.10/site-packages")

if user_site not in sys.path:
    sys.path.insert(0, user_site)

In [4]:
from datasets import load_dataset
ds = load_dataset("deepcopy/DonkeySmall-OCR-Cyrillic-Printed-8", split="train", streaming=True)
print(next(iter(ds))['text'])

детергент


In [1]:
dataset_name = "ljhwild/easy-grayscale-captcha200x40p-240k-plain-various"
output_dir = "./captcha200x40p"
os.system(f"kaggle datasets download -d {dataset_name} -p {output_dir}")
zip_path = os.path.join(output_dir, f"{dataset_name.split('/')[-1]}.zip")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)
os.remove(zip_path)

print(f"Dataset downloaded to: {output_dir}")
print("Contents:", os.listdir(output_dir))

100%|█████████▉| 1.18G/1.19G [00:42<00:00, 33.0MB/s]

100%|██████████| 1.19G/1.19G [00:42<00:00, 29.8MB/s]


✅ Dataset downloaded to: ./captcha200x40p
Contents: ['easy_grayscale_no_effects_system_fonts']


In [18]:

root_dir = "/home/jupyter/project/captcha200x40p/easy_grayscale_no_effects_system_fonts/"
def extract_letters_from_txt(txt_path):
    """Read .txt file, find TEXT= line, return only letters (Latin + Cyrillic)."""
    with open(txt_path, "r", encoding="utf-8") as f:
        first_line = f.readline()[5:].strip()
        return first_line


data = []

print("Scanning folders for .png + .txt pairs...")
from pathlib import Path

root = Path(root_dir)
data = []

for png_path in root.rglob("*.png"):
    txt_path = png_path.with_suffix(".txt")
    if txt_path.exists():
        text = extract_letters_from_txt(str(txt_path))
        if text:
            data.append({
                "image": str(png_path),
                "text": text,
                "source": png_path.parent.name
            })
df_captcha = pd.DataFrame(data)
df_captcha = df_captcha.sample(frac=1, random_state=42).reset_index(drop=True)
output_csv = "captha_word_dataset.csv"
df_captcha.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Saved DataFrame to: {output_csv}")
print("\nFirst 5 rows:")
print(df_captcha.head())

Scanning folders for .png + .txt pairs...
💾 Saved DataFrame to: captha_word_dataset.csv

First 5 rows:
                                               image  ...                 source
0  /home/jupyter/project/captcha200x40p/easy_gray...  ...  10ch_numbers_specials
1  /home/jupyter/project/captcha200x40p/easy_gray...  ...  10ch_numbers_specials
2  /home/jupyter/project/captcha200x40p/easy_gray...  ...      5ch_special_chars
3  /home/jupyter/project/captcha200x40p/easy_gray...  ...      5ch_special_chars
4  /home/jupyter/project/captcha200x40p/easy_gray...  ...  10ch_numbers_specials

[5 rows x 3 columns]


In [ ]:
 def load_image_from_path(img_path):
        return Image.open(img_path).convert("RGB")
def create_document(output,df_word,df_captha,directory = "./train/",num_documents = 1,min_words = 500,max_words = 600,min_captha = 5,max_captcha = 10,canvas_w =800 ,canvas_h = 2500):
    metadata =  []
    for doc_id in range(1, num_documents + 1):
        word_count = random.randint(min_words, max_words)
        captha_count = random.randint(min_captha, max_captcha)

        captha_sample = df_captha.sample(n=captha_count)
        word_sample = df_word.sample(n=word_count)
        df_combined = pd.concat([captha_sample, word_sample], ignore_index=True).dropna().sample(frac=1).reset_index(drop=True)
        
        full_text = full_text = " ".join(df_combined['text'].astype(str))

        document = Image.new("RGB", (canvas_w, canvas_h), (255, 255, 255))
    
    
        x = x_margin
        y = y_start
        current_line_height = 0
        max_line_width = canvas_w - 2 * x_margin
    
        images = df_combined['image']
        
        for i, word_img in enumerate(images):
        
            scale_factor = 0.3
            new_w = int(word_img.width * scale_factor)
            new_h = int(word_img.height * scale_factor)
            word_img = word_img.resize((new_w, new_h), Image.Resampling.LANCZOS)
            w, h = word_img.size
            if x + w > x_margin + max_line_width:
                x = x_margin
                y += current_line_height + line_spacing
                current_line_height = 0
                if i % random.randint(5, 10) == 0 and random.random() < 0.3:
                if y + 100 > canvas_h:
                    break
            document.paste(word_img, (x, y))
            current_line_height = max(current_line_height, h)
        final_height = min(y + current_line_height + 30, canvas_h)
        document = document.crop((0, 0, canvas_w, final_height))
        filename = f"document_{doc_id:05d}_train.jpg"
        image_path = os.path.join(directory, filename)
        document.save(image_path, optimize=True, quality=90)
        metadata.append({
            "image_path": image_path,
            "text": full_text,
        })

    df = pd.DataFrame(metadata)
    df.to_csv(output, index=False, encoding="utf-8")
    print(f"CSV saved to: metadata.csv")
    print(f"Dataset shape: {df.shape}")
    print(f"Example text:\n{df['text'].iloc[0][:200]}...")

In [ ]:
login()

In [3]:
from datasets import load_dataset
ds = load_dataset("deepcopy/DonkeySmall-OCR-Cyrillic-Printed-8", split="train", streaming=True)
subset = ds.take(500000) 

In [4]:
df_captha = pd.read_csv("/home/jupyter/project/captha_word_dataset.csv")

In [5]:
df_captha['image'] = df_captha['image'].apply(load_image_from_path)

In [ ]:
df_word = pd.DataFrame({
            "image": subset['image'],
            "text": subset['text'],
            "source": "donkey"
        })

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a5aa7702-ab4d-488e-b711-5e12768de31b)')' thrown while requesting GET https://huggingface.co/datasets/deepcopy/DonkeySmall-OCR-Cyrillic-Printed-8/resolve/15cb45de49339ffaed2158e832259da96da4cd18/data/train-00000-of-00003.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 745f4dfa-f6d0-4d3b-ac45-848978b5bb21)')' thrown while requesting GET https://huggingface.co/datasets/deepcopy/DonkeySmall-OCR-Cyrillic-Printed-8/resolve/15cb45de49339ffaed2158e832259da96da4cd18/data/train-00001-of-00003.parquet
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f2054a7d-0a2c-49d8-8624-63ba23dad56f)')' thrown while requesting GET https://huggingface.co/datasets/dee

In [39]:
create_document("metadata_big_train.csv",df_word[['image',"text"]],df_captha[['image',"text"]],directory = "./train_Big/",num_documents = 7000)

✅ CSV saved to: metadata.csv
📊 Dataset shape: (7000, 2)
📝 Example text:
ПУБЛИЦИСТАМИ близится ОТКАЗАМИ купят УКЛАДЧИКОВ слэшеры Массагетов сейшелах ЭЭРО Экономичны Прерываниях Усатый гидродинамической дельфах внушаемых вольск нагревающихся филолог йеллоустонским аксиомати...


In [40]:
create_document("metadata_big_val.csv",df_word[['image',"text"]],df_captha[['image',"text"]],directory = "./val_Big/",num_documents = 2000)

✅ CSV saved to: metadata.csv
📊 Dataset shape: (2000, 2)
📝 Example text:
ворончихин ТОКСИНАМ Военком Псам весенний излишнему неньки камминс Чика седьмому замереть Тарантула Пеннер невзрачного Трехминутных Повысит дёрнул дублинский автоконцерна локальное болгарку ПЕРЕСТРАХО...
